# 第7章 支持向量机

> SVM, support vector machine:
> 
> * 线性可分SVM = Hard-margin
> * 线性SVM = Soft-margin
> * 非线性SVM = Soft-margin + Kernel trick

## 一. 线性可分SVM与Hard-margin最大化

> 线性可分支持向量机: Linear seperable SVM，适用于线性可分的训练集，否则无解。
> 
> 硬间隔最大化: Hard-margin SVM，要求SVM对所有实例分类正确，不能有一个错分。

### 1. 线性可分SVM

考虑二元分类问题(binary classification)，$y \in \mathcal{Y} = \{+1,-1\}$。

输入空间(input space)和特征空间(feature space)不同。前者是欧式空间或离散集合，后者是欧式空间或希尔伯特空间。input space与feature space中的元素是__一一对应__的：一个input，对应一个feature space中的feature vector。SVM是在feature space中进行学习(可以理解为feature transformation)。所以，我们现在所说的训练数据集$T$，指代的是feature space。

假设：$T$线性可分。

SVM通过学习，将在feature space中找到一个__分离超平面(seperating hyperplane)__。这个分离超平面满足两个性质：

* 将所有的实例正确分类；
* 与训练数据有最大的间隔，即largest margin(暂时先不管这个词的含义)。

是唯一的。

该largest-margin seperting hyperplane记做$w^*·x+b^*=0$；相应的，分类决策函数为$f(x)=sign(w^*·x+b^*)$。

__那什么是margin? 什么是largest margin?__

![](.\pic\ch7-1.png)

(在线性可分的假设下)满足“将所有实例正确分类的”分离超平面有无数个(上图中便有三个)。它们的$E_{in}$都是0，看起来没有差别。但是直觉会告诉我们，右边的那条线好一些，因为该hyperplane与训练数据“看起来”比较远，因此能够容忍更大的杂讯，也就更加robust，不易overfit。

我们可以这样理解一个hyperplane的margin：从该hyperplane开始，逐渐往外“长”，“长”多少会碰到第一个实例——这里，“长”的多少，就是margin——margin = distance to closest $x_n$。从上图可以看出，最右边的hyperplane，有着最大的margin，因此它的distance to closest $x_n$最大(灰色地方的宽度)。

更加正式一些，我们将定义__函数间隔(functional margin)__与__几何间隔(geometric margin)__。

### 2. 函数间隔 & 几何间隔

假设有一个分离超平面$w·x+b=0$，我们可以用$(w,b)$来表示。一个点与分离超平面越近，我们对该平面给予它的分类越不那么确信，同时$|w·x+b|$会越小；一个点与分离超平面越大，我们会愈加确信该平面给予它的分类，同时$|w·x+b|$越大——__一个点与分离超平面的远近，可以表示分类预测的确信程度__。同时，由于$|w·x+b|$能够相对应的表示点与分离超平面的远近(越大则越远，越小则越近)，因此$|w·x+b|$也能够表示对该点分类的确信程度。另外，如果该点被分类正确，则$y$应与$w·x+b$同号，否则异号。综上，$y(w·x+b)$既能表示分类的正确性，又能表示确信度。

因此，我们这样定义函数间隔：

1. 超平面$(w,b)$与实例$(x_i,y_i)$的函数间隔为：

$$\hat{\gamma}_i = y_i(w·x_i+b)$$

2. 超平面$(w,b)$与数据集$T$的函数间隔为：

$$\hat{\gamma} = \underset{i=1,2,\cdots,N}{min}\hat{\gamma}_i$$

但函数间隔有一个问题，即如果成比例地改变$w$与$b$(如$(w,b)→(\lambda w,\lambda b)$)，超平面并没有改变，因为$w·x+b=0$与$\lambda w·x+\lambda b=0$其实是一个超平面，但函数间隔却会改变，变为原来的$\lambda$倍。

因此，最大化间隔，不能直接最大化函数间隔，否则没有意义(无解)——$w$和$b$都取无穷大。需要给予一定的约束，如$||w||=1$，相当于__标准化__。这时，我们可以定义__几何间隔__：

1. 超平面$(w,b)$与实例$(x_i,y_i)$的几何间隔为：

$$\gamma_i = \frac{y_i(w·x_i+b)}{||w||}$$

2. 超平面$(w,b)$与数据集$T$的函数间隔为：

$$\gamma = \underset{i=1,2,\cdots,N}{min}\gamma_i$$

参考点到平面的距离公式，超平面与实例的几何间隔表示的是实例点到超平面的__带符号的距离__：绝对值表示距离大小，符号表示是否分类正确。

### 3. 间隔最大化: largest margin

线性可分SVM最大化的，是__几何间隔__，即：求解能够正确划分训练数据集(hard-margin)并且几何间隔最大(largest-margin)的分离超平面。这样唯一的超平面的最大优点，是它对于最难分的实例点(离超平面最近的点)，也有足够大的确信度将其分开。

我们将上述问题数学化表示：

$$\underset{w,b}{max}\ \gamma$$

$$s.t.\ \frac{y_i(w·x_i+b)}{||w||} \ge \gamma,\quad i=1,2,\cdots,N$$

因为函数间隔与几何间隔存在如下关系：

$$\gamma=\frac{\hat{\gamma}}{||w||}$$

我们将上述代入原最优化问题，得到等价问题：

$$\underset{w,b}{max}\ \frac{\hat{\gamma}}{||w||}$$

$$s.t.\ y_i(w·x_i+b) \ge \hat{\gamma},\quad i=1,2,\cdots,N$$

由于函数间隔的大小，并不影响原最优化问题，因此我们可以随意设置函数间隔大小，得到的最优化问题仍然和原问题等价。那么不妨设$\hat{\gamma}=1$：

$$\underset{w,b}{max}\ \frac{1}{||w||}$$

$$s.t.\ y_i(w·x_i+b) \ge 1,\quad i=1,2,\cdots,N$$

稍微修改一下目标函数，将最大化转化为最小化：

$$\underset{w,b}{min}\ \frac{1}{2}||w||^2$$

$$s.t.\ y_i(w·x_i+b) \ge 1,\quad i=1,2,\cdots,N$$

上述最优化问题，即为线性可分SVM的最优化问题——也是一个__凸二次规划问题(convex quadratic programming)__。

\begin{align*}
              & \underset{x \in \mathbf{R}^n}{min}\ \ f(x)\\
    s.t.\ \ \ & c_i(x) \le 0,\quad i=1,2,\cdots,k\\
              & h_j(x) = 0,\quad j=1,2,\cdots,l
\end{align*}

__凸优化问题__：

* $f(x)$与$c_i(x)$都是连续可微的凸函数；
* $h_j(x)$是仿射函数——$f(x)=a·x+b$，那么$f(x)$就是仿射函数。

__凸二次规划问题__：

* 凸优化问题；
* $f(x)$是二次函数；
* $c_i(x)$是仿射函数。

有专门解决凸二次规划问题的程序，十分容易解决。

### 4. 支持向量

落在边界上的点，被称为support vector(candidates)。它们满足：

$$|y_i(w·x_i+b)|=1$$

两条边界分别为：

$$y(w·x+b)=1$$
$$y(w·x+b)=-1$$

两条边界之间不会有任何实例点，他们的距离为:

$$\frac{2}{||w||}$$

训练数据集中的支持向量决定着唯一的那个最大间隔的超平面，其他实例点并不重要：

* 去掉非支持向量，解不变；
* 去掉支持向量，解可能改变。

### 5. 对偶问题

为什么我们往往不解原问题，而解对偶问题？

* dual往往更容易求解；
* 能够使用kernel trick。

Recap: primal problem

$$\underset{w,b}{min}\ \frac{1}{2}||w||^2$$

$$s.t.\ y_i(w·x_i+b) \ge 1,\quad i=1,2,\cdots,N$$

引入拉格朗日乘子，构造拉格朗日函数：

$$L(w,b,\alpha) = \frac{1}{2}||w||^2+\sum_{i=1}^N\alpha_i\Big(1-y_i(w·x_i+b)\Big),\quad \alpha_i \ge 0$$

primal problem等价于：

$$\underset{w,b}{min}\ \underset{\alpha;\alpha_i\ge0}{max}\ L(w,b,\alpha)$$

其dual problem为：

$$\underset{\alpha;\alpha_i\ge0}{max}\ \underset{w,b}{min}\ L(w,b,\alpha)$$

我们可以继续化简dual problem：对于内层的最小化问题，我们分别对$\nabla_w L$与$\nabla_b L$置零，得到：

$$w=\sum_{i=1}^N \alpha_iy_ix_i$$

$$\sum_{i=1}^N \alpha_iy_i=0$$

将上面两个等式代入$L$中，可以将dual problem转化为：

\begin{align*}
              & \underset{\alpha}{max}\ -\frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_j(x_i·x_j)+\sum_i\alpha_i\\
    s.t.\ \ \ & \alpha_i \ge 0,\quad i=1,2,\cdots,N\\
              & \sum_i\alpha_iy_i=0
\end{align*}

上述问题被称为__standard hard-margin SVM dual__(已被证明，SVM中的primal与dual满足强对偶关系，是等价的，因此我们解dual即可)。

解得$\alpha^*$后，我们可以运用KKT条件，解$w^*$与$b^*$。

由$\nabla_w L=0$得：

$$w^* = \sum_i\alpha_i^*y_ix_i$$

注意到KKT条件中的：

$$\alpha_i^*\Big(1-y_i(w^*·x_i+b^*)\Big)=0,\quad i=1,2,\cdots,N$$

可以证明，至少存在一个$\alpha_j>0$：(反证法)若$\alpha^*=0$，则$w^*=0$，不成立。那么对应的，有：

$$1-y_j(w^*·x_j+b^*)=0$$

得：

$$b^* = y_j - w^*x_j = y_j - \sum_i\alpha_i^*y_ix_i·x_j$$

最后，我们再次定义支持向量：我们将$\alpha^*_i>0$对应的样本点$(x_i,y_i)$称为support vector。根据互补条件，我们知道，support vector一定在边界上(但在边界上的样本点不一定是support vector)。

## 二. 线性SVM与Soft-margin最大化

> 但由于noise的存在，我们的训练数据集往往并不是线性可分的；
>
> 并且，即使线性可分，如果我们坚持将资料分开而不犯一个错，往往会因noise而造成overfit。

### 1. 初步改进

Recap: primal problem

$$\underset{w,b}{min}\ \frac{1}{2}||w||^2$$

$$s.t.\ y_i(w·x_i+b) \ge 1,\quad i=1,2,\cdots,N$$

尝试改造一下上面的最优化问题，使得我们放弃对一些点分类正确的要求。可以：

\begin{align*}
              & \underset{w,b}{min}\ \frac{1}{2}||w||^2+C\sum_i I\Big(y_i \ne sign(w·x_i+b)\Big)\\
    s.t.\ \ \ & \ y_i(w·x_i+b) \ge 1,\quad for\ correct\ i\\
              & \ y_i(w·x_i+b) \ge -\infty,\quad for\ incorrect\ i
\end{align*}

上述最优化问题中，$C$后面的求和项表示__犯错点的总数__，而$C$类似“调节旋钮”: trade off of __large margin__ & __noise tolerance__

* large $C$：对犯错点数量的惩罚大，因此模型倾向于少犯错，即使margin需要小一些——易过拟合；
* small $C$：对犯错点数量的惩罚小，因此模型倾向于多犯错，使margin大一些——易欠拟合。

还可以继续改写成：

\begin{align*}
              & \underset{w,b}{min}\ \frac{1}{2}||w||^2+C\sum_i I\Big(y_i \ne sign(w·x_i+b)\Big)\\
    s.t.\ \ \ & \ y_i(w·x_i+b) \ge 1-\infty·I\Big(y_i \ne sign(w·x_i+b)\Big),\quad i=1,2,\cdots,N\\
\end{align*}

但是上述最优化问题有两个缺点：
1. 非QP问题；
2. 无法区分large error与small error。

### 2. 引入松弛变量

\begin{align*}
              & \underset{w,b,\xi}{min}\ \frac{1}{2}||w||^2+C\sum_i \xi_i\\
    s.t.\ \ \ & \ y_i(w·x_i+b) \ge 1-\xi_i,\quad i=1,2,\cdots,N\\
        \ \ \ & \xi_i \ge 0,\quad i=1,2,\cdots,N\\
\end{align*}

此前我们要求对于每一个样本$(x_i,y_i)$，其必须位于边界上或边界外且不能分类错误，即必须满足$y_i(w·x_i+b) \ge 1$。但现在，我们给每一个样本配置一个松弛变量$\xi_i \ge 0$，不强求$y_i(w·x_i+b) \ge 1$，只需要$y_i(w·x_i+b) \ge 1-\xi_i$即可。这说明，该样本点可以位于边界内部，甚至可以分类错误，这取决于$\xi_i$的大小：

* $\xi_i$大，说明犯的错越大；
* $\xi_i$大，说明犯的错越小；

而把$\xi_i$的和作为最小化项目的一部分，说明我们还是尽量要分类正确+位于边界上或外，也就是我们还是要尽量使$\xi_i$小的。

注意到，引入松弛变量后的最优化问题，依然是凸二次规划问题，$w$的解唯一，但$b$的解可能不唯一，而是存在于一个区间。

### 3. 线性SVM

解引入松弛变量$\xi$后的最优化问题：

\begin{align*}
              & \underset{w,b,\xi}{min}\ \frac{1}{2}||w||^2+C\sum_i \xi_i\\
    s.t.\ \ \ & \ y_i(w·x_i+b) \ge 1-\xi_i,\quad i=1,2,\cdots,N\\
        \ \ \ & \xi_i \ge 0,\quad i=1,2,\cdots,N\\
\end{align*}

得到的$(w^*,b^*)$，称为线性SVM，也叫做Soft-Margin SVM。

### 4. 对偶问题

拉格朗日函数：

$$L(w,b,\xi,\alpha,\beta)=\frac{1}{2}||w||^2+C\sum_i\xi_i-\sum_i\alpha_i\Big(y_i(w·x_i+b)-1+\xi_i\Big)-\sum_i\beta_i\xi_i$$

__primal__:

$$\underset{w,b,\xi}{min}\ \underset{\alpha,\beta;\alpha_i\ge0,\beta_i\ge0}{max}\ L(w,b,\xi,\alpha,\beta)$$

__dual__:

$$\underset{\alpha,\beta;\alpha_i\ge0,\beta_i\ge0}{max}\ \underset{w,b,\xi}{min}\ L(w,b,\xi,\alpha,\beta)$$

同样，强对偶成立，等价，存在$(w^*,b^*,\xi^*,\alpha^*,\beta^*)$。

对于内层最小化问题，将下面三式分别置零：

* $\nabla_wL$: $w=\sum_i \alpha_iy_ix_i$
* $\nabla_bL$: $\sum_i \alpha_iy_i=0$
* $\nabla_{\xi_i}L$: $\alpha_i+\beta_i=C$

带回dual，化简得：

\begin{align*}
              & \underset{\alpha,\beta}{max}\ -\frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_j(x_i·x_j)+\sum_i\alpha_i\\
    s.t.\ \ \ & \alpha_i \ge 0,\quad i=1,2,\cdots,N\\
              & \beta_i \ge 0,\quad i=1,2,\cdots,N\\
              & \alpha_i+\beta_i = C,\quad i=1,2,\cdots,N\\
              & \sum_i\alpha_iy_i=0\\
              & \Big(w=\sum_i\alpha_iy_ix_i\Big)
\end{align*}

将第二个约束和第三个约束结合，消去$\beta$，得到最终的dual：

\begin{align*}
              & \underset{\alpha}{min}\ \frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_j(x_i·x_j)-\sum_i\alpha_i\\
    s.t.\ \ \ & 0 \le \alpha_i \le C,\quad i=1,2,\cdots,N\\
              & \sum_i\alpha_iy_i=0\\
              & \Big(w=\sum_i\alpha_iy_ix_i\Big)
\end{align*}

KKT条件：

* $\alpha^*_i\Big(y_i(w^*·x_i+b)-1+\xi_i^*\Big) = 0$
* $\alpha^*_i \ge 0$
* $y_i(w^*·x_i+b)-1+\xi_i^* \ge 0$

* $\beta_i^*\xi_i^* = 0$
* $\beta_i^*\ge 0$
* $\xi_i^*\ge 0$

若存在$0 < \alpha^*_j < C$，则根据$\alpha_j+\beta_j=C$，有$\beta_j > 0$；因此$\xi_j=0$且$y_i(w^*·x_i+b)-1+\xi_i^*=0$；得$b^* = y_j - \sum_iy_i\alpha^*_j(x_i·x_j)$

### 5. 支持向量

我们还是从$\alpha$的视角来看support vector:

* Non-SV，即$\alpha_i^*=0$，的样本点：
    * 根据$\alpha_i+\beta_i=C$，$\beta^*_i$大于零，所以$\xi_i^*=0$；
    * 由于$\alpha_i^*=0$，不能确认$y_i(w^*·x_i+b)-1=0$；
    * 没有违反margin，在margin外或上；
* Free-SV，即$0<\alpha_i^*<C$的样本点：
    * 根据$\alpha_i+\beta_i=C$，$\beta^*_i$大于零，所以$\xi_i^*=0$；
    * 由于$\alpha_i^*>0$，可以确认$y_i(w^*·x_i+b)-1=0$；
    * 没有违反margin，在margin上；
* Bounded-SV，即$\alpha_i^*=C$的样本点：
    * 根据$\alpha_i+\beta_i=C$，$\beta^*_i$等于零；
    * 所以$\xi_i^* \ge 0$；
    * 有违反margin，或在margin上。

### 6. 合页损失函数

Soft-Margin SVM中的$\xi$被称为margin violation，它记录的是某个点犯错(违背margin)的情况：

* 若没有犯错，则$\xi_i=0$，此时$y_i(w·x_i+b)\ge1$，因此$1-y_i(w·x_i+b)\le0$；
* 若犯错，甚至到了对面，则$\xi_i=1-y_i(w·x_i+b)>0$，此时$y_i(w·x_i+b)<1$，因此$1-y_i(w·x_i+b)>0$；

综上：

$$\xi_i = max\Big(1-y_i(w·x_i+b),0\Big)$$

Soft-Margin SVM的原始问题:

\begin{align*}
              & \underset{w,b,\xi}{min}\ \frac{1}{2}||w||^2+C\sum_i \xi_i\\
    s.t.\ \ \ & \ y_i(w·x_i+b) \ge 1-\xi_i,\quad i=1,2,\cdots,N\\
        \ \ \ & \xi_i \ge 0,\quad i=1,2,\cdots,N\\
\end{align*}

等价于如下的无约束最优化问题:

$$\underset{b,w}{min}\ \frac{1}{2}||w||^2+C\sum_i max\Big(1-y_i(w·x_i+b),0\Big)$$

上述最优化问题的目标函数，类似于，__L2正则化__：

$$min\ \frac{\lambda}{N}||w||^2+\frac{1}{N}\sum err$$

因此，我们可以将Soft-Margin SVM看做以$max\Big(1-y_i(w·x_i+b),0\Big)$为single lost function的结构风险(L2)最小化。这个$max$函数，又被称为__合页损失函数__。

至此，我们将Soft-margin SVM与以特定$\hat{err}$的L2正则化联系了起来：

* large margin = fewer hyperplanes = L2 reg of short $w$
* larger $C$ = smaller $\lambda$ = less reg

## 三. 非线性SVM与核函数

### 1. Motivation

对于__训练集(输入空间)非线性可分__的情况，我们往往采取的是__非线性变换(Non-linear transformation)__，将输入空间的训练集映射到特征空间(往往是更高维度的)，然后在特征空间训练一个线性模型，如SVM，进行分类。

这也是我们在一开始就强调的，我们的SVM是在特征空间中训练的，而非输入空间。

为了区分，我们用$x$表示输入空间中的样例，用$z$表示特征空间中的样例，$z = \phi(x)$。此外，我们记$z$的维度为$\tilde{d}$($x$的维度为$n$)，且一般来说$n<<\tilde{d}$。

这个很大的$\tilde{d}$非常烦人，我们把问题从primal转化为dual，就是因为$\tilde{d}$太大了，对应的QP问题不好解——$\tilde{d}+1$个变量，$N$个约束；转成dual后，变成了$N$个变量，$N+1$个约束的QP问题——看似通过将primal转成dual，我们将$\tilde{d}$消掉了！

然而，在解dual的时候，我们要向解QP的程式传入矩阵$Q_D$，其元素$q_{n,m}=y_ny_mz_n^Tz_m$——要算$z_n^Tz_m$，还是$\tilde{d}$维的运算……

因为$z_n^Tz_m = \phi(x_n)^T\phi(x_m)$，所以计算$z_n^Tz_m$实际上是两步：

1. 非线性转换——麻烦，要转化到很高很高的维度；
2. 算内积——麻烦，维度很大很大的内积；

能不能将两步结合在一起，直接在低维度进行运算？Kernel Trick的思想。

我们以2次转换为例，进行探索(因为有平方，所以这里我们把维度的角标变成下标)：

$\Phi_2(x) = \Big( 1,x_1, x_2, \cdots, x_d, x_1^2, x_1x_2, \cdots, x_1x_d, x_2x_1,x_2^2,\cdots,x_2x_d,\cdots,x_d^2\Big)$

假设有两个点$x$与$x'$

\begin{align*}
    \Phi_2(x)^T\Phi_2(x')	& = 1+\sum_{i=1}^d x_ix_i'+\sum_{i=1}^d\sum_{j=1}^dx_ix_jx_i'x_j'\\
                            & = 1+\sum_{i=1}^d x_ix_i'+\sum_{i=1}^dx_ix_i'\sum_{j=1}^dx_jx_j'\\
                            & = 1+x^Tx'+(x^Tx')^2\\
\end{align*}

可见，我们好像只需要进行低维度的内积运算，就可以得到原本必须经过两步高维度操作才能得到的结果——Kernel = Transform + Inner product——两步合成一步！

__Kernel Function:__

$$K_{\Phi}(x,x') = \Phi(x)^T\Phi(x')$$

一个Kernel Function对应着一个转换(但不是唯一的)，但我们可以不用关心这个转换，直接用此核函数计算原来要计算的$z_n^Tz_m$：例如，针对2次转换，原来，我们是要先把每个样本进行2次转换(费时1)，然后再成对成对算内积(费时2)；现在，我们直接用核函数，先成对算低维度内积，然后进行$1+a+a^2$的“数字运算”即可——省时：

* 不用算$z$
* 不用存$w$

$$z_n^Tz_m = \phi(x_n)^T\phi(x_m) = K(x_n,x_m)$$

用在SVM中：

$$q_{n,m} = y_ny_mz_n^Tz_m = y_ny_mK(x_n,x_m)$$

\begin{align*}
    b & = y_s - w^Tz_s\\
      & = y_s - \Big(\sum_{i=1}^N\alpha_iy_iz_i\Big)^Tz_s\\
      & = y_s - \sum_{i=1}^N\alpha_iy_iK(x_i,x_s)\\
\end{align*}

\begin{align*}
    g_{SVM}(x) & = sign\Big(w^T\phi(x)+b\Big)\\
               & = sign\Big(\sum_{i=1}^N\alpha_iy_i\phi(x_i)^T\phi(x)+b\Big)\\
               & = sign\Big(\sum_{i=1}^N\alpha_iy_iK(x_i,x)+b\Big)\\
\end{align*}

### 2. Polynomial Kernel

Gernel Polynomial Kernel ($\gamma > 0,\ \zeta\ge0$):

$$K_Q(x,x')=(\zeta+\gamma x^Tx')^Q$$

其背后对应了一个$\Phi_Q$，但不重要了。

Linear Kernel ($\gamma = 0,\ \zeta = 0,\ Q=1$):

$$K_1(x,x')=x^Tx'$$

相当于没有$\Phi$。

### 3. Gaussian Kernel

又称为RBF核(Radial Basis Function Kernel):

$$K(x,x')=exp(-\gamma||x-x'||^2)$$

\begin{align*}
    g_{SVM}(x) & = sign\Big(\sum_{SV}\alpha_iy_iK(x_i,x)+b\Big)\\
               & = sign\Big(\sum_{SV}\alpha_iy_iexp(-\gamma||x_i-x||^2)+b\Big)\\
\end{align*}

可见，高斯核的SVM，实质上是一个个以support vector为中心的高斯分布的线性组合；而$\gamma$越大，对应的Gaussian分布越尖；因此，$\gamma$越大，模型越容易overfit。

### 4. Kernel的比较

1. Linear Kernel: $K(x,x')=x^Tx'$
    * 【优】safe，不易过拟合；
    * 【优】fast，计算量小；
    * 【优】explainable，能拿到$w$；
    * 【缺】欠拟合；
2. Polynomial Kernel
    * 【优】拟合能力强一些；
    * 【缺】Q很大时，数值计算困难；
    * 【缺】有三个参数要调……
3. Gaussian Kernel
    * 【优】拟合能力更强了；
    * 【优】只有一个参数；
    * 【缺】拿不到$w$；
    * 【缺】比linear慢，计算量稍大；
    * 【缺】过拟合风险；
4. Self-designed Kernel
    * $K$是核函数的充要条件：①对称函数；②对任意数据集$D^N$，矩阵$K^{N×N}:K_{m,n}=K(x_m,x_n)$是半正定的。

## 四. 序列最小最优化算法

序列最小最优化算法，Sequential Minimal Optimization，是针对求解SVM的dual开发出的算法。当然，SVM的dual是一个凸二次规划问题，可以使用传统的QP算法求解。但是，当__训练样本的容量非常大__时，传统的QP算法会非常缓慢。而SMO算法，是专门为SVM设计更加高效的算法。

SMO要解决的是Soft-margin svm dual:

\begin{align*}
              & \underset{\alpha}{min}\ \frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_jK(x_i,x_j)-\sum_i\alpha_i\\
    s.t.\ \ \ & 0 \le \alpha_i \le C,\quad i=1,2,\cdots,N\\
              & \sum_i\alpha_iy_i=0\\
\end{align*}

SMO的基本思路：

* 如果所有的$\alpha_i$都满足KKT条件，则$\alpha$就是该最优化问题的解；
* 否则，选择两个变量，如$\alpha_1$与$\alpha_2$，固定其他变量，针对这两个变量构建一个__子二次规划问题__——可以通过解析法求解；
    * 两个变量的选择：第一个变量，是违反KKT条件最严重的那一个；第二个变量，是由约束条件自动确定的；

### 1. 两个变量的二次规划问题

假设选择的两个变量是$\alpha_1,\alpha_2$，其他变量$\alpha_i(i=1,2,\cdots,N)$是固定的。于是我们可以得到如下的子二次规划问题(将原目标函数中完全不含$\alpha_1$和$\alpha_2$的项目删去)：

\begin{align*}
              & \underset{\alpha_1,\alpha_2}{min}\ W(\alpha_1,\alpha_2)=\frac{1}{2}K_{11}\alpha_1^2+\frac{1}{2}K_{22}\alpha_2^2+y_1y_2K_{12}\alpha_1\alpha_2-(\alpha_1+\alpha_2)+y_1\alpha_1\sum_{i=3}^Ny_i\alpha_iK_{i1}+y_2\alpha_2\sum_{i=3}^Ny_i\alpha_iK_{i2}\\   
    s.t.\ \ \ & 0 \le \alpha_i \le C,\quad i=1,2\\
              & \alpha_1y_1+\alpha_2y_2 = -\sum_{i=3}^N\alpha_iy_i=\zeta\\
\end{align*}

其中，$K_{ij}=K(x_i,x_j)$，$\zeta$是常数。

因为仅有两个变量$\alpha_1$与$\alpha_2$，所以上述子问题的约束可以用二维空间的图形表示：

![](./pic/ch7-2.png)

不等式约束将$\alpha_1$与$\alpha_2$框在边长为$C$的正方形内；等式约束使$\alpha_1$与$\alpha_2$位于平行与盒子对角线的直线上。两者结合，则把$\alpha_1$与$\alpha_2$约束在__一条平行于对角线的线段上__。

因为确定了$\alpha_1$与$\alpha_2$中的一个，另一个可以由等式约束求出，因此该问题实质上也是单变量最优化问题——不妨考虑$\alpha_2$为主变量。

假设子问题的__初始可行解(上一轮的解，所以叫old)__为$(\alpha_1^{old},\alpha_2^{old})$，最优解为$(\alpha_1^{new},\alpha_2^{new})$，沿着约束方向但不受盒子约束时(在直线上，而不一定必须在线段上)$\alpha_2$的最优解为$\alpha_2^{new,unc}$。

1. 首先，最优解$\alpha_2^{new}$肯定要满足不等式约束(被框在盒子里)，因此$\alpha_2^{new}$需要满足($L$和$H$是$\alpha^{new}_2$所在的对角线端点的界)：

$$L \le \alpha_2^{new} \le H$$

若$y_1 \ne y_2$：

$$L=max(0,\alpha_2^{old}-\alpha_1^{old})$$
$$H=min(C,C+\alpha_2^{old}-\alpha_1^{old})$$

若$y_1 = y_2$：

$$L=max(0,\alpha_2^{old}+\alpha_1^{old}-C)$$
$$H=min(C,\alpha_2^{old}+\alpha_1^{old})$$

2. 接着，我们求出沿着约束方向但不受盒子约束时(在直线上，而不一定必须在线段上)$\alpha_2$的最优解为$\alpha_2^{new,unc}$。记(书上)：

$$g(x) = \sum_{i=1}^N\alpha_iy_iK(x_i,x)+b$$

(但我认为，最好写成)：

$$g^{old}(x) = \sum_{i=1}^N\alpha_i^{old}y_iK(x_i,x)+b^{old}$$

令(书上)：

$$E_i = g(x_i)-y_i,\quad i=1,2$$

(同样)：

$$E_i^{old} = g^{old}(x_i)-y_i,\quad i=1,2$$

此外，为了更加方便地表示子规划问题的目标函数，我们再引入(书上的写法会引入混淆)：

$$v_i^{old} = \sum_{j=3}^Ny_j\alpha_iK_{ij}=g^{old}(x_i)-\sum_{j=1}^2\alpha_j^{old}y_iK_{ij}-b^{old},\quad i=1,2$$

易知，$v_i^{old}$与$\alpha_1,\alpha_2$无关，因为$\alpha_1^{old},\alpha_2^{old}$是固定的。道这样，子规划问题的目标函数可以转化为：

$$W(\alpha_1,\alpha_2)=\frac{1}{2}K_{11}\alpha_1^2+\frac{1}{2}K_{22}\alpha_2^2+y_1y_2K_{12}\alpha_1\alpha_2-(\alpha_1+\alpha_2)+y_1\alpha_1v_1^{old}+y_2\alpha_2v_2^{old}$$

因为$\alpha_1y_1+\alpha_2y_2=\zeta$且$y_i^2=1$，所以我们可以将$\alpha_1$用$\alpha_2$表示：

$$\alpha_1 = (\zeta-y_2\alpha_2)y_1$$

将上式代入$W(\alpha_1,\alpha_2)$，消去$\alpha_1$。然后将$W(\alpha_2)$对$\alpha_2$求偏导置零(注意代入$v_i^{old}$，并使用$\alpha_1^{old}y_1+\alpha_2^{old}y_2=\zeta$)，解得：

$$\alpha_2^{new,unc} = \alpha_2^{old} + \frac{y_2(E_1^{old}-E_2^{old})}{K_{11}+K_{22}-2K_{12}}$$

再简化一下：

$$\alpha_2^{new,unc} = \alpha_2^{old} + \frac{y_2(E_1^{old}-E_2^{old})}{\eta}$$

其中，

$$\eta = K_{11}+K_{22}-2K_{12} = ||\Phi(x_1)-\Phi(x_2)||^2$$

3. 最后，综合最开始的$L \le \alpha_2^{new} \le H$，得到：

\begin{equation}
    \alpha_2^{new}=
    \begin{cases}
    H,                  & \alpha_2^{new,unc}>H,\\
    \alpha_2^{new,unc}, & L \le \alpha_2^{new,unc} \le H\\
    L,                  & \alpha_2^{new,unc}<L,\\
    \end{cases}
\end{equation}

由$\alpha_1^{old}y_1+\alpha_2^{old}y_2 = \alpha_1^{new}y_1+\alpha_2^{new}y_2$得：

$$\alpha_1^{new} = \alpha_1^{old}+y_1y_2(\alpha_2^{old}-\alpha_2^{new})$$

4. 结束，得到$(\alpha_1^{new},\alpha_2^{new})$。

### 2. 变量的选择方法

两个变量的选择：第一个，严重违反KKT；第二个，根据第一个选择。

#### 1) 第一个变量的选择

SMO的术语：__外层循环__——在训练样本中选取违反KKT条件最严重的样本点。具体的，检验样本点$(x_i,y_i)$：

* 若$\alpha_i=0$，检查$g(x_i)\ge1$
* 若$0<\alpha_i<C$，检查$g(x_i)=1$
* 若$\alpha_i=C$，检查$g(x_i)\le1$

另外，先遍历所有满足$0<\alpha_i<C$的样本点，即在margin上的；若这些free SV都满足，则遍历整个训练集。

#### 2) 第二个变量的选择

SMO的术语：__内层循环__——已经找到了第一个变量$\alpha_1$，内层循环是找$\alpha_2$；选择标准是，希望$\alpha_2$有足够大的变化。

由上一部分的推导可知，$\alpha^{new}_2$依赖于$|E_1^{old}-E_2^{old}|$。一种简单的选择方法是，选择这样的$\alpha_2$，使其对应的$|E_1^{old}-E_2^{old}|$最大(因为$\alpha_1$已经确定，也就是那个样本确定了，所以$E_1^{old}$也确定了)：

* 如果$E_1^{old}$是正的，那么选择最小的$E_i^{old}$作为$E_2^{old}$；
* 如果$E_1^{old}$是负的，那么选择最大的$E_i^{old}$作为$E_2^{old}$；

特殊情况下，如果内层循环选到的$\alpha_2$不能使目标函数有足够的下降，那么换一种方式继续选择其他的$\alpha_2$：

* 遍历margin上的free SV，依次将其作为$\alpha_2$，直到拿到一个使得目标函数有足够的下降；
* 若找不到合适的，则遍历整个数据集；
* 如果还没有，则放弃第一个变量，重选$\alpha_1$。

#### 3) 选完两个变量之后

①重新计算b；②更新$E_i$列表。

①重新计算b：

(书上的假设：更新完$\alpha_1$与$\alpha_2$后，他们对应的样本点不再违反KKT条件，所以可以用KKT条件算新的$b$。)

当$0<\alpha_1^{new}<C$时，由$y_1g(x_1)=1$知：

$$y_1 = \sum_{i=1}^{N}\alpha^{new/old}_iy_iK_{i1}+b^{new}_1=\alpha_1^{new}y_1K_{11}+\alpha_2^{new}y_2K_{12}+\sum_{i=3}^{N}\alpha^{old}_iy_iK_{i1}+b^{new}_1$$

又：

$$E_1^{old} = g^{old}(x_1)-y_1$$

联立得：

$$b_1^{new} = -E_1^{old}-y_1K_{11}(\alpha_1^{new}-\alpha_1^{old})-y_2K_{21}(\alpha_2^{new}-\alpha_2^{old})+b^{old}$$

同理，若$0<\alpha_2^{new}<C$：

$$b_2^{new} = -E_2^{old}-y_2K_{12}(\alpha_1^{new}-\alpha_1^{old})-y_2K_{22}(\alpha_2^{new}-\alpha_2^{old})+b^{old}$$

若$0<\alpha_1^{new}<C$与$0<\alpha_2^{new}<C$同时成立，那么$b_1^{new}=b_2^{new}$。若其中一个是0或C，则$b_1^{new} \ne b_2^{new}$，两者之间的所有b值都是符合KKT的，我们选择他们的__中点__。

②更新$E_i$列表：

$$E_i^{new} = g^{new}(x_i)-y_i = \sum_{i=1}^N\alpha_i^{new}y_iK(x_i,x)+b^{new}-y_i,\quad i=1,2$$

当然：

$$\sum_{i=1}^N = \sum_{SV}$$